In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import tensorflow as tf
from tensorflow import keras

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__)
    print(module.__version__)

sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)
matplotlib
3.1.2
numpy
1.17.4
pandas
0.25.3
sklearn
0.22
tensorflow
2.0.0
tensorflow_core.keras
2.2.4-tf


## 1.tf.function and autograph

auto-graph是实现tf.function的内部机制

### 1.1 tf.function()

用python编写的函数并没有像tensorflow库里的库函数一样被优化，运行时比优化过的库函数在性能上有所欠缺，而
tf.function则是将用python编写函数转为库函数，使其得以优化

In [2]:
# convert: py func ---> keras func
# Method 1:
def scaled_elu(z, scale=1.0, alpha=1.0):
    # function: z >= 0 ? scale * z : scaled * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)  # z > 0.0
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

scaled_elu_tf = tf.function(scaled_elu) # 重点
print(scaled_elu_tf(-3.))
print(scaled_elu_tf(tf.constant([-3, -2.5])))

# Method 2:
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment = increment / 2.0
    return total

print(converge_to_2(20))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(1.9999981, shape=(), dtype=float32)


In [3]:
# 性能比较测试， 在有gpu的时候加速差异会更加明显
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

15.8 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.5 ms ± 82.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
# 从库函数找回原python编写的普通函数
origin_func = scaled_elu_tf.python_function
print(origin_func is scaled_elu)

True


### 1.2 tf.autograph()

In [5]:
# 这个函数的功能是将python编写的函数转为keras库中的函数后，对应的库函数代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown("```python\n{}\n```".format(code)))
    
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

## 2. 函数中的变量

在函数中不能声明变量，但可以声明常量，可能是因为在编写神经网络时变量更多要在开始初始化

In [6]:
var = tf.Variable(0.)
@tf.function
def add_21():
#     var = tf.Variable(0.) 变量写在这里不行
    return var + 21 # var.assign_add(21)
print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [7]:
@tf.function
def cube(z):
    return tf.pow(z, 3)

print(cube(tf.constant([1., 2., 3.])))
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


## 3. 函数签名（参数类型、参数个数）

作用：
1. 限制数据的输入类型；
2. 保存模型的条件之一；

In [8]:
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name="x")])
def cube(z):
    return tf.pow(z, 3)

try:
    print(cube(tf.constant([1., 2., 3.])))  # inputs are float
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1, 2, 3]))), print()

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)



(None, None)

函数的作用：
1. @tf.function: py func ---> keras func；
2. get_concrete_function: keras func --- add input_signature ---> SavedMode(图定义)

In [9]:
cube_func_int32 = cube.get_concrete_function(
                                    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)
print(cube_func_int32 is cube.get_concrete_function(
                                    tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(
                                    tf.constant([1,2,3], tf.int32)))

True
True


In [10]:
cube_func_int32.graph

In [11]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [12]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(type(pow_op))
print(pow_op)

<class 'tensorflow.python.framework.ops.Operation'>
name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [13]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [14]:
cube_func_int32.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [15]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [16]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}